# Receiving All
## Example 4. Pub/Sub with Direct exchange using routing key.
- Sender generates logs with types 'error', 'info', 'warning'
- First Receiver accepts all of log types.
- Second Receiver accepts only 'error' log type.

In [2]:
#!/usr/bin/env python
import pika
import sys

EXCHANGE_NAME = 'direct_logs'

connection = pika.BlockingConnection(pika.ConnectionParameters(
        host='192.168.99.100'))
channel = connection.channel()

channel.exchange_declare(exchange=EXCHANGE_NAME,
                         type='direct')

severities = ['info', 'warning', 'error']



## Bindings

In previous examples we were already creating bindings. You may recall code like:



In [3]:
# channel.queue_bind(exchange=exchange_name,
#                   queue=queue_name)


A binding is a relationship between an exchange and a queue. This can be simply read as: **the queue is interested in messages from this exchange.**

Bindings can take an extra routing_key parameter. To avoid the confusion with a basic_publish parameter we're going to call it a **binding key**. This is how we could create a binding with a key:

In [4]:
# channel.queue_bind(exchange=exchange_name,
#                   queue=queue_name,
#                   routing_key='black')


The meaning of a binding key depends on the exchange type. The fanout exchanges, which we used previously, simply ignored its value.



#### Multiple bindings
![](https://www.rabbitmq.com/img/tutorials/direct-exchange-multiple.png)
It is perfectly legal to bind multiple queues with the same binding key. In our example we could add a binding between X and Q1 with binding key black. In that case, the direct exchange will behave like fanout and will broadcast the message to all the matching queues. A message with routing key black will be delivered to both Q1 and Q2.



## Subscribing
Receiving messages will work just like in the previous tutorial, with one exception - we're going to create a new binding for each severity we're interested in.



In [5]:
result = channel.queue_declare(exclusive=True)
queue_name = result.method.queue

for severity in severities:
    channel.queue_bind(exchange=EXCHANGE_NAME,
                       queue=queue_name,
                       routing_key=severity)

![](https://www.rabbitmq.com/img/tutorials/python-four.png)


In [6]:
print ' [*] Waiting for logs. To exit press CTRL+C'

def callback(ch, method, properties, body):
    print " [x] %r:%r" % (method.routing_key, body,)

channel.basic_consume(callback,
                      queue=queue_name,
                      no_ack=True)

channel.start_consuming()

 [*] Waiting for logs. To exit press CTRL+C
 [x] 'error':'Hello World!'
 [x] 'error':'Hello World!'
 [x] 'info':'Hello World!'
 [x] 'info':'Hello World!'
 [x] 'warning':'Hello World!'
 [x] 'error':'Hello World!'
 [x] 'error':'Hello World!'
 [x] 'info':'Hello World!'
 [x] 'info':'Hello World!'
 [x] 'warning':'Hello World!'
 [x] 'error':'Hello World!'
 [x] 'error':'Hello World!'
 [x] 'info':'Hello World!'
 [x] 'info':'Hello World!'
 [x] 'warning':'Hello World!'
 [x] 'error':'Hello World!'
 [x] 'error':'Hello World!'
 [x] 'info':'Hello World!'
 [x] 'info':'Hello World!'
 [x] 'warning':'Hello World!'
 [x] 'error':'Hello World!'


KeyboardInterrupt: 

In [7]:
connection.close()